# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

``` sql
 {{ config(materialized='table') }}

SELECT 
  COUNT(*) AS order_count, 
  EXTRACT(YEAR FROM order_approved_at) AS year 
FROM 
  `dbt-shop-willian.dbt_shop.orders` 
GROUP BY 
  year
```

    Comando terminal
    
``dbt run --models shop.orders_by_year`` 

![Questão 1](image_atividade3/questao1.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

```sql
  {{ config(materialized='view') }}

SELECT 
  count(*) as total_pedido, 
  c.customer_state as estado 
FROM 
  `dbt_shop.customers` c 
INNER JOIN `dbt_shop.orders` o 
  ON c.customer_id = o.customer_id 
GROUP BY
  c.customer_state
```

    Comando terminal
    
``dbt run --models shop.total_orders`` 

![Questão 2](image_atividade3/questao2.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

```sql
{{ config(materialized='view') }}

SELECT * FROM `dbt-shop-willian.dataset_willian.total_orders`
WHERE total_pedido > {{ var("number_of_orders") }}
```


    Comando terminal
    
``dbt run --models shop.number_of_orders --vars "{'number_of_orders': 5000}"`` 

![Questão 3](image_atividade3/questao3.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

```sql
{{ config(materialized='view') }}

SELECT 
  CASE 
    WHEN customer_state  IN ('AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO') THEN 'Norte'
    WHEN customer_state  IN ('MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA') THEN 'Nordeste'
    WHEN customer_state  IN ('MG', 'ES', 'RJ', 'SP') THEN 'Sudeste'
    WHEN customer_state  IN ('PR', 'SC', 'RS') THEN 'Sul'
    ELSE 'Outra região'
  END AS regiao,
  COUNT(*) AS cliente
FROM `dbt-shop-willian.dbt_shop.customers`
GROUP BY
 regiao

```


    Comando terminal
    
``dbt run --models shop.customers_by_region`` 

![Questão 4](image_atividade3/questao4_.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

```sql
{{ config(materialized='table') }}  

SELECT 
  product_id, 
  shipping_limit_date, 
  (price + freight_value) as preco_total 
FROM `dbt-shop-willian.dbt_shop.items` 
```

    Comando terminal
    
``dbt run --models shop.items_costs`` 

![Questão 6](image_atividade3/questao5.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

```sql
{{ config(materialized='view') }}

SELECT 
  items.order_id,
  orders.order_status,
  (orders.val_total_pedido + items.freight_value) AS valor_total
FROM 
  `dbt-shop-willian.dbt_shop.items` items 
  INNER JOIN (
    SELECT
      o.order_id, 
      o.order_status, 
      SUM(i.price) AS val_total_pedido
    FROM 
      `dbt-shop-willian.dbt_shop.orders` o 
      INNER JOIN `dbt-shop-willian.dbt_shop.items` i 
        ON i.order_id = o.order_id 
    GROUP BY 
      o.order_id, 
      o.order_status   
    HAVING 
      o.order_status = 'shipped'
  ) orders 
    ON items.order_id = orders.order_id
```

    Comando terminal
    
``dbt run --models shop.orders_shipped_costs`` 

![Questao 6](image_atividade3/questao6.png)